# Advanced Querying Mongo

Importing libraries and setting up connection

In [3]:
from pymongo import MongoClient
client = MongoClient("localhost:27017")
db = client.get_database("ironhack")

In [5]:
c = db.get_collection("companies")

In [6]:
c

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'ironhack'), 'companies')

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [13]:
list(c.find({"name": "Babelgum"},{'name':1,'_id':0}))

[{'name': 'Babelgum'}]

### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [17]:
list(c.find({'number_of_employees': {"$gt": 5000}},{'name': 1,'number_of_employees': 1, '_id':0}).sort('number_of_employees', -1).limit(20))

[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000},
 {'name': 'Sony', 'number_of_employees': 180500},
 {'name': 'LG', 'number_of_employees': 177000},
 {'name': 'Ford', 'number_of_employees': 171000},
 {'name': 'Boeing', 'number_of_employees': 160000},
 {'name': 'Digital Equipment Corporation', 'number_of_employees': 140000},
 {'name': 'Nokia', 'number_of_employees': 125000},
 {'name': 'MItsubishi Electric', 'number_of_employees': 107000}

### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [35]:
list(c.find({"$and": [{'founded_year': {"$gte": 2000}},{'founded_year': {"$lte": 2004}}]},{'name': 1,'founded_year': 1, '_id':0}).limit(10))

[{'name': 'Digg', 'founded_year': 2004},
 {'name': 'Facebook', 'founded_year': 2004},
 {'name': 'StumbleUpon', 'founded_year': 2002},
 {'name': 'Gizmoz', 'founded_year': 2003},
 {'name': 'Plaxo', 'founded_year': 2002},
 {'name': 'Technorati', 'founded_year': 2002},
 {'name': 'AddThis', 'founded_year': 2004},
 {'name': 'Veoh', 'founded_year': 2004},
 {'name': 'Meetup', 'founded_year': 2002},
 {'name': 'SmugMug', 'founded_year': 2002}]

### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [26]:
list(c.find({"$and": [{'ipo.valuation_amount': {"$gt": 100000000}},{'founded_year': {"$lt": 2010}}]},{'name': 1,'ipo.valuation_amount': 1, '_id':0}).limit(20))

[{'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Yelp', 'ipo': {'valuation_amount': 1300000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Brightcove', 'ipo': {'valuation_amount': 290000000}},
 {'name': 'KIT digital', 'ipo': {'valuation_amount': 235000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'OpenTable', 'ipo': {'valuation_amount': 1050000000}},
 {'name': 'ChannelAdvisor', 'ipo': {'valuation_amount': 287000000}},
 {'name': 'Jive Software', 'ipo': {'valuation_amount': 1000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Wix', 'ipo': {'valuation_amount': 750000000}},
 {'name': 'Shutterfly', 'ipo': {'valuation_amount': 350000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'Salesforce', 'ipo': {'

### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [22]:
list(c.find({"$and": [{'founded_year': {"$lt": 2005}},{'number_of_employees': {"$lt": 100}}]},{'name': 1,'number_of_employees': 1, 'founded_year': 1,'_id':0}).sort('number_of_employees',-1).limit(10))

[{'name': 'Fulcrum Analytics',
  'number_of_employees': 99,
  'founded_year': 1993},
 {'name': 'PeoplePC', 'number_of_employees': 98, 'founded_year': 1999},
 {'name': 'Clinnovo Research Labs Pvt Ltd',
  'number_of_employees': 95,
  'founded_year': 2004},
 {'name': 'eG Innovations', 'number_of_employees': 95, 'founded_year': 2001},
 {'name': 'Kineto Wireless', 'number_of_employees': 95, 'founded_year': 2001},
 {'name': 'Pure Digital Technologies',
  'number_of_employees': 95,
  'founded_year': 2001},
 {'name': 'Volagratis', 'number_of_employees': 95, 'founded_year': 2004},
 {'name': 'picoChip', 'number_of_employees': 92, 'founded_year': 2000},
 {'name': 'Tableau Software', 'number_of_employees': 92, 'founded_year': 2003},
 {'name': 'CompareNetworks', 'number_of_employees': 90, 'founded_year': 2000}]

### 6. All the companies that don't include the `partners` field.

In [25]:
list(c.find({"partners": {"$exists": False}}))

[]

### 7. All the companies that have a null type of value on the `category_code` field.

In [34]:
list(c.find({"category_code": None},{'name': 1}).limit(10))

[{'_id': ObjectId('52cdef7c4bab8bd6752980f6'), 'name': 'Collective'},
 {'_id': ObjectId('52cdef7c4bab8bd675298225'), 'name': 'Snimmer'},
 {'_id': ObjectId('52cdef7c4bab8bd675298226'), 'name': 'KoolIM'},
 {'_id': ObjectId('52cdef7c4bab8bd675298261'), 'name': 'Level9 Media'},
 {'_id': ObjectId('52cdef7c4bab8bd675298262'), 'name': 'VidKing'},
 {'_id': ObjectId('52cdef7c4bab8bd67529826e'), 'name': 'Drigg'},
 {'_id': ObjectId('52cdef7c4bab8bd675298276'), 'name': 'SpaceTime'},
 {'_id': ObjectId('52cdef7c4bab8bd675298279'), 'name': 'Touch Clarity'},
 {'_id': ObjectId('52cdef7c4bab8bd67529827a'), 'name': 'MMDAYS'},
 {'_id': ObjectId('52cdef7c4bab8bd67529827b'), 'name': 'Inside Group'}]

### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [32]:
list(c.find({"$and": [{'number_of_employees': {"$gte": 100}},{'number_of_employees': {"$lt": 1000}}]},{'name': 1,'number_of_employees': 1, '_id':0}).limit(20))

[{'name': 'AdventNet', 'number_of_employees': 600},
 {'name': 'AddThis', 'number_of_employees': 120},
 {'name': 'OpenX', 'number_of_employees': 305},
 {'name': 'LifeLock', 'number_of_employees': 644},
 {'name': 'Jajah', 'number_of_employees': 110},
 {'name': 'Livestream', 'number_of_employees': 120},
 {'name': 'Ustream', 'number_of_employees': 250},
 {'name': 'iContact', 'number_of_employees': 300},
 {'name': 'Yelp', 'number_of_employees': 800},
 {'name': 'Dailymotion', 'number_of_employees': 120},
 {'name': 'RockYou', 'number_of_employees': 106},
 {'name': 'Meebo', 'number_of_employees': 200},
 {'name': 'Eventbrite', 'number_of_employees': 200},
 {'name': 'Box', 'number_of_employees': 950},
 {'name': 'Conduit', 'number_of_employees': 215},
 {'name': 'Redfin', 'number_of_employees': 100},
 {'name': 'oDesk', 'number_of_employees': 120},
 {'name': 'Simply Hired', 'number_of_employees': 100},
 {'name': 'PhotoBox', 'number_of_employees': 600},
 {'name': 'Spreadshirt', 'number_of_employees'

### 9. Order all the companies by their IPO price in a descending order.

In [37]:
list(c.find({},{'name': 1,'ipo.valuation_amount': 1, '_id':0}).sort('ipo.valuation_amount',-1).limit(20))

[{'name': 'GREE', 'ipo': {'valuation_amount': 108960000000}},
 {'name': 'Facebook', 'ipo': {'valuation_amount': 104000000000}},
 {'name': 'Amazon', 'ipo': {'valuation_amount': 100000000000}},
 {'name': 'Twitter', 'ipo': {'valuation_amount': 18100000000}},
 {'name': 'Groupon', 'ipo': {'valuation_amount': 12800000000}},
 {'name': 'Tencent', 'ipo': {'valuation_amount': 11000000000}},
 {'name': 'Western Digital', 'ipo': {'valuation_amount': 9430000000}},
 {'name': 'LinkedIn', 'ipo': {'valuation_amount': 9310000000}},
 {'name': 'BMC Software', 'ipo': {'valuation_amount': 6000000000}},
 {'name': 'Rackspace', 'ipo': {'valuation_amount': 5440000000}},
 {'name': 'Baidu', 'ipo': {'valuation_amount': 4000000000}},
 {'name': 'TripAdvisor', 'ipo': {'valuation_amount': 3273770000}},
 {'name': 'HomeAway', 'ipo': {'valuation_amount': 3000000000}},
 {'name': 'Zillow', 'ipo': {'valuation_amount': 2550000000}},
 {'name': 'Nielsen', 'ipo': {'valuation_amount': 1600000000}},
 {'name': 'Yelp', 'ipo': {'valu

### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [39]:
list(c.find({},{'name': 1,'number_of_employees': 1, '_id':0}).sort('number_of_employees',-1).limit(10))


[{'name': 'Siemens', 'number_of_employees': 405000},
 {'name': 'IBM', 'number_of_employees': 388000},
 {'name': 'Toyota', 'number_of_employees': 320000},
 {'name': 'PayPal', 'number_of_employees': 300000},
 {'name': 'Nippon Telegraph and Telephone Corporation',
  'number_of_employees': 227000},
 {'name': 'Samsung Electronics', 'number_of_employees': 221726},
 {'name': 'Accenture', 'number_of_employees': 205000},
 {'name': 'Tata Consultancy Services', 'number_of_employees': 200300},
 {'name': 'Flextronics International', 'number_of_employees': 200000},
 {'name': 'Safeway', 'number_of_employees': 186000}]

### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [43]:
list(c.find({'founded_month':{'$gt':6}},{'name':1, '_id':0,'founded_month':1}).limit(10))

[{'name': 'Wetpaint', 'founded_month': 10},
 {'name': 'Zoho', 'founded_month': 9},
 {'name': 'Digg', 'founded_month': 10},
 {'name': 'Omnidrive', 'founded_month': 11},
 {'name': 'eBay', 'founded_month': 9},
 {'name': 'Joost', 'founded_month': 10},
 {'name': 'Plaxo', 'founded_month': 11},
 {'name': 'Powerset', 'founded_month': 10},
 {'name': 'Kyte', 'founded_month': 12},
 {'name': 'Thoof', 'founded_month': 12}]

### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [46]:
list(c.find({"$and": [{'founded_year': {"$lt": 2000}},{'acquisition.price_amount': {"$gt": 10000}}]},{'name': 1,'founded_year': 1, '_id':0, 'acquisition.price_amount':1}).limit(10))

[{'name': 'Postini',
  'founded_year': 1999,
  'acquisition': {'price_amount': 625000000}},
 {'name': 'SideStep',
  'founded_year': 1999,
  'acquisition': {'price_amount': 180000000}},
 {'name': 'Recipezaar',
  'founded_year': 1999,
  'acquisition': {'price_amount': 25000000}},
 {'name': 'Cyworld',
  'founded_year': 1999,
  'acquisition': {'price_amount': 7140000}},
 {'name': 'PayPal',
  'founded_year': 1998,
  'acquisition': {'price_amount': 1500000000}},
 {'name': 'Snapfish',
  'founded_year': 1999,
  'acquisition': {'price_amount': 300000000}},
 {'name': 'Neopets',
  'founded_year': 1999,
  'acquisition': {'price_amount': 160000000}},
 {'name': 'Sun Microsystems',
  'founded_year': 1982,
  'acquisition': {'price_amount': 7400000000}},
 {'name': 'Zappos',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1200000000}},
 {'name': 'Alibaba',
  'founded_year': 1999,
  'acquisition': {'price_amount': 1000000000}}]

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [49]:
list(c.find({'acquisition.acquired_year': {"$gt": 2010}},{'name': 1, '_id':0, 'acquisition':1}).limit(5))

[{'name': 'Wetpaint',
  'acquisition': {'price_amount': 30000000,
   'price_currency_code': 'USD',
   'term_code': 'cash_and_stock',
   'source_url': 'http://allthingsd.com/20131216/viggle-tries-to-bulk-up-its-social-tv-business-by-buying-wetpaint/?mod=atdtweet',
   'source_description': ' Viggle Tries to Bulk Up Its Social TV Business by Buying Wetpaint',
   'acquired_year': 2013,
   'acquired_month': 12,
   'acquired_day': 16,
   'acquiring_company': {'name': 'Viggle', 'permalink': 'viggle'}}},
 {'name': 'Digg',
  'acquisition': {'price_amount': 500000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2012/07/12/betaworks-acquires-digg/',
   'source_description': 'Betaworks Acquires Digg (TechCrunch)',
   'acquired_year': 2012,
   'acquired_month': 7,
   'acquired_day': 12,
   'acquiring_company': {'name': 'betaworks', 'permalink': 'betaworks'}}},
 {'name': 'Geni',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
 

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [52]:
list(c.find({},{'name': 1, '_id':0, 'founded_year':1}).sort('founded_year',-1).limit(5))

[{'name': 'Advaliant', 'founded_year': 2013},
 {'name': 'Fixya', 'founded_year': 2013},
 {'name': 'iBazar', 'founded_year': 2013},
 {'name': 'Fluc', 'founded_year': 2013},
 {'name': 'Wamba', 'founded_year': 2013}]

### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [55]:
list(c.find({'founded_day': {"$lte": 7}},{'name': 1, '_id':0, 'founded_day':1}).sort('acquisition.price_amount',-1).limit(10))

[{'name': 'Netscape', 'founded_day': 4},
 {'name': 'PayPal', 'founded_day': 1},
 {'name': 'Zappos', 'founded_day': 1},
 {'name': 'Alibaba', 'founded_day': 1},
 {'name': 'Postini', 'founded_day': 2},
 {'name': 'Danger', 'founded_day': 1},
 {'name': 'Clearwell Systems', 'founded_day': 6},
 {'name': 'PrimeSense', 'founded_day': 1},
 {'name': 'Amobee', 'founded_day': 1},
 {'name': 'BlueLithium', 'founded_day': 1}]

### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [57]:
list(c.find({"$and": [{'category_code': 'web'},{'number_of_employees': {"$gt": 4000}}]},{'name': 1,'number_of_employees': 1,'category_code':1, '_id':0}).sort('number_of_employees',1))

[{'name': 'Expedia', 'category_code': 'web', 'number_of_employees': 4400},
 {'name': 'AOL', 'category_code': 'web', 'number_of_employees': 8000},
 {'name': 'Webkinz', 'category_code': 'web', 'number_of_employees': 8657},
 {'name': 'Rakuten', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Los Angeles Times Media Group',
  'category_code': 'web',
  'number_of_employees': 10000},
 {'name': 'Groupon', 'category_code': 'web', 'number_of_employees': 10000},
 {'name': 'Yahoo!', 'category_code': 'web', 'number_of_employees': 13600},
 {'name': 'eBay', 'category_code': 'web', 'number_of_employees': 15000},
 {'name': 'Experian', 'category_code': 'web', 'number_of_employees': 15500}]

### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [60]:
list(c.find({"$and": [{'acquisition.price_currency_code': 'EUR'},{'acquisition.price_amount': {"$gt": 10000000}}]},{'name': 1,'acquisition.price_amount': 1,'acquisition.price_currency_code':1, '_id':0}))

[{'name': 'ZYB',
  'acquisition': {'price_amount': 31500000, 'price_currency_code': 'EUR'}},
 {'name': 'Apertio',
  'acquisition': {'price_amount': 140000000, 'price_currency_code': 'EUR'}},
 {'name': 'Greenfield Online',
  'acquisition': {'price_amount': 40000000, 'price_currency_code': 'EUR'}},
 {'name': 'Webedia',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'Wayfinder',
  'acquisition': {'price_amount': 24000000, 'price_currency_code': 'EUR'}},
 {'name': 'Tuenti Technologies',
  'acquisition': {'price_amount': 70000000, 'price_currency_code': 'EUR'}},
 {'name': 'BioMed Central',
  'acquisition': {'price_amount': 43400000, 'price_currency_code': 'EUR'}}]

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [61]:
list(c.find({'founded_month':{'$lt':4}},{'name':1, '_id':0,'acquisition':1}).limit(10))

[{'name': 'Facebook', 'acquisition': None},
 {'name': 'Twitter', 'acquisition': None},
 {'name': 'StumbleUpon',
  'acquisition': {'price_amount': 29000000,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://techcrunch.com/2009/04/13/ebay-unacquires-stumbleupon/',
   'source_description': "StumbleUpon Beats Skype In Escaping EBay's Clutches",
   'acquired_year': 2009,
   'acquired_month': 4,
   'acquired_day': None,
   'acquiring_company': {'name': 'StumbleUpon', 'permalink': 'stumbleupon'}}},
 {'name': 'Gizmoz',
  'acquisition': {'price_amount': None,
   'price_currency_code': 'USD',
   'term_code': None,
   'source_url': 'http://www.gizmoz.com',
   'source_description': 'TechCrunch',
   'acquired_year': 2009,
   'acquired_month': 12,
   'acquired_day': 15,
   'acquiring_company': {'name': 'Daz 3d', 'permalink': 'daz-3d'}}},
 {'name': 'Helio',
  'acquisition': {'price_amount': 39000000,
   'price_currency_code': 'USD',
   'term_code': 'stock',
   'source_ur

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [77]:

list(c.find ({'$and':[{"acquisition.acquired_year": {'$lt': 2011}}, {'founded_year': { '$gte' : 2000 , '$lte' : 2010 }}]}, {"_id":0, "acquisition.acquired_year":1,'founded_year':1}).limit(10))

[{'founded_year': 2002, 'acquisition': {'acquired_year': 2009}},
 {'founded_year': 2003, 'acquisition': {'acquired_year': 2009}},
 {'founded_year': 2005, 'acquisition': {'acquired_year': 2008}},
 {'founded_year': 2006, 'acquisition': {'acquired_year': 2009}},
 {'founded_year': 2002, 'acquisition': {'acquired_year': 2008}},
 {'founded_year': 2006, 'acquisition': {'acquired_year': 2008}},
 {'founded_year': 2004, 'acquisition': {'acquired_year': 2010}},
 {'founded_year': 2005, 'acquisition': {'acquired_year': 2009}},
 {'founded_year': 2003, 'acquisition': {'acquired_year': 2005}},
 {'founded_year': 2005, 'acquisition': {'acquired_year': 2006}}]

### 20. All the companies that have been 'deadpooled' after the third year.

In [103]:

b=list(c.find({'deadpooled_year':{'$exists': True}},{'deadpooled_year':1,'founded_year': 1,'name':1, '_id':0}))
lista=[]
for i in b:

    if i['deadpooled_year'] != None and i['founded_year']!=None:    
        if i['deadpooled_year'] < 4:
            lista.append(i)
        elif (i['deadpooled_year']-i['founded_year']) < 4:
            lista.append(i)



In [104]:
lista


[{'name': 'Wetpaint', 'founded_year': 2005, 'deadpooled_year': 1},
 {'name': 'AdventNet', 'founded_year': 1996, 'deadpooled_year': 2},
 {'name': 'Zoho', 'founded_year': 2005, 'deadpooled_year': 3},
 {'name': 'Omnidrive', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Sparter', 'founded_year': 2007, 'deadpooled_year': 2008},
 {'name': 'Jangl SMS', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'Spotplex', 'founded_year': 2007, 'deadpooled_year': 2008},
 {'name': 'TripUp', 'founded_year': 2006, 'deadpooled_year': 2008},
 {'name': 'Fleck', 'founded_year': 2006, 'deadpooled_year': 2008},
 {'name': 'Zooomr', 'founded_year': 2006, 'deadpooled_year': 2006},
 {'name': 'TripHub', 'founded_year': 2005, 'deadpooled_year': 2008},
 {'name': 'adBrite', 'founded_year': 2003, 'deadpooled_year': 2001},
 {'name': 'WebbAlert', 'founded_year': 2007, 'deadpooled_year': 2009},
 {'name': 'KnockaTV', 'founded_year': 2007, 'deadpooled_year': 2008},
 {'name': 'EventVue', 'founded_year': 2